In [88]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

## **Training**

In [89]:
# Load and prepare dataset for SVR
df = pd.read_csv('output/data_merged_cleaned.csv')

# Convert date and sort chronologically for time-based split
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')
df = df.sort_values('date').reset_index(drop=True)

# Separate features and target
X = df.drop(columns=['aqi_pm2.5', 'date'])
y = df['aqi_pm2.5']

# Time-based split (80% train, 20% test)
split_idx = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")
print(f"Date range: {df['date'].iloc[0].strftime('%d/%m/%Y')} to {df['date'].iloc[-1].strftime('%d/%m/%Y')}")

Training samples: 1172
Testing samples: 293
Date range: 01/06/2019 to 29/11/2023


In [90]:
# Scale features for SVR
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).ravel()

In [91]:
# Train SVR with RBF kernel
svr_model = SVR(kernel='rbf')
svr_model.fit(X_train_scaled, y_train_scaled)

# Make predictions
y_train_pred_scaled = svr_model.predict(X_train_scaled)
y_test_pred_scaled = svr_model.predict(X_test_scaled)

# Convert back to original scale
y_train_pred = scaler_y.inverse_transform(y_train_pred_scaled.reshape(-1, 1)).ravel()
y_test_pred = scaler_y.inverse_transform(y_test_pred_scaled.reshape(-1, 1)).ravel()

# Calculate metrics
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

In [92]:
print("SVR MODEL PERFORMANCE:")
print(f"RMSE: {test_rmse:.2f}")
print(f"R² Score: {test_r2:.4f}")

SVR MODEL PERFORMANCE:
RMSE: 38.34
R² Score: 0.7085


## **Hyperparamater Turning**

In [93]:
# Define parameter grid for SVR tuning
param_grid = {
    'C': [0.1, 1, 10, 100],
    'epsilon': [0.01, 0.1, 0.2, 0.5],
    'gamma': ['scale', 'auto', 0.01, 0.1]
}

# TimeSeriesSplit for time-based cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# GridSearchCV for SVR
grid_search = GridSearchCV(
    SVR(kernel='rbf'),
    param_grid,
    cv=tscv,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1
)

grid_search.fit(X_train_scaled, y_train_scaled)

,estimator,SVR()
,param_grid,"{'C': [0.1, 1, ...], 'epsilon': [0.01, 0.1, ...], 'gamma': ['scale', 'auto', ...]}"
,scoring,'neg_root_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,TimeSeriesSpl...est_size=None)
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,kernel,'rbf'


In [94]:
# Get best model and parameters
best_svr = grid_search.best_estimator_
best_params = grid_search.best_params_

print("BEST SVR PARAMETERS:")
for param, value in best_params.items():
    print(f"{param}: {value}")

BEST SVR PARAMETERS:
C: 10
epsilon: 0.1
gamma: 0.01


In [95]:
# Make predictions with tuned model
y_test_pred_tuned = scaler_y.inverse_transform(
    best_svr.predict(X_test_scaled).reshape(-1, 1)
).ravel()

# Calculate metrics for tuned model
test_rmse_tuned = np.sqrt(mean_squared_error(y_test, y_test_pred_tuned))
test_r2_tuned = r2_score(y_test, y_test_pred_tuned)
test_mae_tuned = mean_absolute_error(y_test, y_test_pred_tuned)

print("TUNED SVR MODEL PERFORMANCE:")
print(f"RMSE: {test_rmse_tuned:.2f}")
print(f"R² Score: {test_r2_tuned:.4f}")
print(f"MAE: {test_mae_tuned:.2f}")

TUNED SVR MODEL PERFORMANCE:
RMSE: 37.83
R² Score: 0.7162
MAE: 27.91


In [96]:

# Ensure required variables exist for SVR category analysis
from sklearn.metrics import mean_squared_error

# Define AQI categories
aqi_bins = [0, 100, 150, 200, 300, 500]
bin_labels = ['Good (0-100)', 'Moderate (101-150)', 'Unhealthy (151-200)', 
              'Very Unhealthy (201-300)', 'Hazardous (301-500)']

# Predict with tuned SVR model
y_pred_svr = scaler_y.inverse_transform(best_svr.predict(X_test_scaled).reshape(-1, 1)).ravel()

# Categorize test samples
y_test_binned = pd.cut(y_test, bins=aqi_bins, labels=bin_labels)

# Calculate performance for SVR
print("SVR PERFORMANCE BY AQI CATEGORY")
print(f"{'Category':<25} {'Samples':<10} {'RMSE':<12} {'Bias':<12}")
print("-" * 60)

category_results_svr = []
for category in bin_labels:
    mask = y_test_binned == category
    if mask.sum() > 0:
        category_rmse = np.sqrt(mean_squared_error(y_test[mask], y_pred_svr[mask]))
        category_bias = (y_test[mask] - y_pred_svr[mask]).mean()
        
        category_results_svr.append({
            'category': category,
            'samples': mask.sum(),
            'rmse': category_rmse,
            'bias': category_bias
        })
        
        print(f"{category:<25} {mask.sum():<10} {category_rmse:<12.2f} {category_bias:<12.2f}")

# Identify best and worst
best_svr = min(category_results_svr, key=lambda x: x['rmse'])
worst_svr = max(category_results_svr, key=lambda x: x['rmse'])

print(f"\nSVR Best: {best_svr['category']} (RMSE: {best_svr['rmse']:.2f})")
print(f"SVR Worst: {worst_svr['category']} (RMSE: {worst_svr['rmse']:.2f})")

SVR PERFORMANCE BY AQI CATEGORY
Category                  Samples    RMSE         Bias        
------------------------------------------------------------
Good (0-100)              38         48.82        -36.69      
Moderate (101-150)        121        22.42        -7.21       
Unhealthy (151-200)       70         29.97        1.41        
Very Unhealthy (201-300)  45         32.27        16.78       
Hazardous (301-500)       19         91.23        85.09       

SVR Best: Moderate (101-150) (RMSE: 22.42)
SVR Worst: Hazardous (301-500) (RMSE: 91.23)


## **Feature Engineering**

In [97]:

df_sorted = df.copy()

# Base features
X_base = df_sorted.drop(columns=['aqi_pm2.5', 'date'])
y = df_sorted['aqi_pm2.5']

# Create enhanced features
X_enhanced = X_base.copy()

# Lag features
X_enhanced['temp_avg_lag1'] = df_sorted['temp_avg_c'].shift(1)
X_enhanced['wind_avg_lag1'] = df_sorted['wind_speed_avg_mph'].shift(1)
X_enhanced['humidity_avg_lag1'] = df_sorted['humidity_avg_percent'].shift(1)

# Interaction features
X_enhanced['wind_temp_interaction'] = X_enhanced['wind_speed_avg_mph'] * X_enhanced['temp_min_c']
X_enhanced['wind_humidity_interaction'] = X_enhanced['wind_speed_avg_mph'] * X_enhanced['humidity_avg_percent']

# High-risk indicators
X_enhanced['high_risk_month'] = X_enhanced['month'].isin([1, 2, 10, 11]).astype(int)
X_enhanced['early_winter'] = ((X_enhanced['month'] == 11) | (X_enhanced['month'] == 12)).astype(int)

# Rolling features
X_enhanced['wind_3day_avg'] = df_sorted['wind_speed_avg_mph'].rolling(3, min_periods=1).mean()
X_enhanced['temp_3day_avg'] = df_sorted['temp_avg_c'].rolling(3, min_periods=1).mean()

# Fill NaN values
X_enhanced = X_enhanced.fillna(X_enhanced.mean())

print(f"Enhanced features created: {X_enhanced.shape[1]} total features")


print(f"\nTraining set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Training dates: {df_sorted['date'].iloc[0].strftime('%d/%m/%Y')} to {df_sorted['date'].iloc[split_idx-1].strftime('%d/%m/%Y')}")
print(f"Testing dates: {df_sorted['date'].iloc[split_idx].strftime('%d/%m/%Y')} to {df_sorted['date'].iloc[-1].strftime('%d/%m/%Y')}")


Enhanced features created: 35 total features

Training set: 1172 samples
Test set: 293 samples
Training dates: 01/06/2019 to 06/01/2023
Testing dates: 07/01/2023 to 29/11/2023


## **Final Evaluation**

In [98]:
from sklearn.svm import SVR
from sklearn.metrics import root_mean_squared_error, r2_score

# Initialize SVR with RBF kernel
svr_model = SVR(kernel='rbf', C=10, epsilon=0.1, gamma='scale')

# Train SVR
svr_model.fit(X_train_scaled, y_train)

# Make predictions
y_pred_svr = svr_model.predict(X_test_scaled)

# Evaluate SVR
rmse_svr = root_mean_squared_error(y_test, y_pred_svr)
r2_svr = r2_score(y_test, y_pred_svr)

print("\nSVR MODEL PERFORMANCE:")
print(f"RMSE: {rmse_svr:.2f}")
print(f"R²: {r2_svr:.4f}")


# Category performance analysis
print("\nSVR PERFORMANCE BY AQI CATEGORY:")

aqi_bins = [0, 100, 150, 200, 300, 500]
bin_labels = ['Good (0-100)', 'Moderate (101-150)', 'Unhealthy (151-200)', 
              'Very Unhealthy (201-300)', 'Hazardous (301-500)']

y_test_binned = pd.cut(y_test, bins=aqi_bins, labels=bin_labels)

print(f"{'Category':<25} {'Samples':<10} {'RMSE':<12} {'Bias':<12}")
print("-" * 60)

for category in bin_labels:
    mask = y_test_binned == category
    if mask.sum() > 0:
        category_rmse = root_mean_squared_error(y_test[mask], y_pred_svr[mask])
        category_bias = (y_test[mask] - y_pred_svr[mask]).mean()
        print(f"{category:<25} {mask.sum():<10} {category_rmse:<12.2f} {category_bias:<12.2f}")




SVR MODEL PERFORMANCE:
RMSE: 41.27
R²: 0.6622

SVR PERFORMANCE BY AQI CATEGORY:
Category                  Samples    RMSE         Bias        
------------------------------------------------------------
Good (0-100)              38         48.99        -42.88      
Moderate (101-150)        121        20.70        -11.36      
Unhealthy (151-200)       70         27.57        -2.39       
Very Unhealthy (201-300)  45         35.65        21.69       
Hazardous (301-500)       19         113.68       107.37      


In [99]:
# Compare with XGBoost
print("\nCOMPARISON WITH XGBOOST:")
print(f"SVR RMSE: {rmse_svr:.2f} vs XGBoost RMSE: 33.69")
print()
# Hazardous days specific analysis
hazardous_mask = y_test > 300
if hazardous_mask.sum() > 0:
    svr_hazardous_rmse = root_mean_squared_error(y_test[hazardous_mask], y_pred_svr[hazardous_mask])
    print(f"\nSVR Hazardous Days RMSE: {svr_hazardous_rmse:.2f}")
    print(f"XGBoost Hazardous RMSE: 57.83")



COMPARISON WITH XGBOOST:
SVR RMSE: 41.27 vs XGBoost RMSE: 33.69


SVR Hazardous Days RMSE: 113.68
XGBoost Hazardous RMSE: 57.83


## **Saving the Model**

In [102]:
import joblib

svr_model_data = {
    'model': svr_model,
    'scaler_X': scaler_X,
    'feature_names': X_train.columns.tolist()
}
joblib.dump(svr_model_data, 'output/models/svr_model.pkl')
print("SVR model saved to output/models/svr_model.pkl")

SVR model saved to output/models/svr_model.pkl
